<a href="https://colab.research.google.com/github/kurek0010/neutral-network/blob/main/03_keras/01_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* @author: krakowiakpawel9@gmail.com  
* @site: e-smartdata.org

### Model sekwencjny (Sequential Model):
1. [Wprowadzenie - pierwszy model](#a0)
2. [Funkcje aktywacji](#a1)
3. [Kompilacja modelu](#a2)
4. [Trenowanie modelu](#a3)
5. [Przykład - klasyfikacja binarna](#a4)
6. [Przykład - klasyfikacja wieloklasowa](#a5)
7. [Przykład - regresja](#a6)



Konfiguracja środowiska


In [ ]:
# Przygotowanie środowiska do pracy z Tensorflow 2.0.
# Jeśli otrzymasz błąd podczas instalacji Tensorflow uruchom tę komórkę raz jeszcze.

##!pip uninstall -y tensorflow
##!pip install -q tensorflow==2.0.0

Found existing installation: tensorflow 2.0.0
Uninstalling tensorflow-2.0.0:
  Successfully uninstalled tensorflow-2.0.0


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
tf.__version__

'2.15.0'

### <a name='a0'></a>Wprowadzenie - pierwszy model
Model sekwencyjny to nic innego jak liniowy stos warstw.

In [2]:
# utworzenie instancji klasy Sequential
from tensorflow.keras.models import Sequential

model = Sequential()
print(model)

Podstawowym elementem składowym modelu są warstwy. Aby dodać najbardziej standardową warstwę - warstwę gęsto połączoną należy użyć warstwy **Dense**. Aby dodać warstwę do modelu należy użyć metody *.add()*

In [3]:
from tensorflow.keras.layers import Dense

model.add(Dense(units=4, input_shape=(10,)))

Wyświetlenie podsumowania modelu: metoda *.summary()*

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
Total params: 44 (176.00 Byte)
Trainable params: 44 (176.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Dodanie kolejnej warstwy

In [5]:
model.add(Dense(units=2))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
 dense_1 (Dense)             (None, 2)                 10        
                                                                 
Total params: 54 (216.00 Byte)
Trainable params: 54 (216.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### <a name='a1'></a>Funkcje aktywacji
Istotnym elemenem sieci neuronowych jest dobór odpowiednich funkcji aktywacji. Funkcje aktywacji jak sama nazwa wskazuje są odpowiedzialne za aktywowanie odpowiednich neuronów podczas procesu uczenia.

Jeżeli nie określimy podczas dodawania warstwy funkcji aktywacji, domyślnie stosowana jest liniowa funkcja aktywacji, tzn. $a(x)=x$

Warstwa z liniową funkcją aktywacji może uczyć się tylko liniowych przekształceń danych wejściowych. Dlatego stosuje się różne funkcje aktywacji aby rozwiazywać problemy nieliniowe.

[Keras: Funkcje aktywacji](https://keras.io/activations/)

In [6]:
from tensorflow.keras.activations import linear

random_data = sorted(np.random.randn(200))
data = pd.DataFrame({'data': random_data, 'linear': linear(random_data)})
data.head()

,data,linear
0,-2.450118,-2.450118
1,-2.061522,-2.061522
2,-2.046942,-2.046942
3,-2.022047,-2.022047
4,-1.972046,-1.972046


In [7]:
px.line(data, x='data', y='linear', width=800, range_y=[-2, 2])

In [8]:
from tensorflow.keras.activations import sigmoid

data = pd.DataFrame({'data': random_data, 'sigmoid': sigmoid(random_data)})
data.head()

,data,sigmoid
0,-2.450118,0.079430
1,-2.061522,0.112893
2,-2.046942,0.114362
3,-2.022047,0.116908
4,-1.972046,0.122169


In [9]:
px.line(data, x='data', y='sigmoid', width=800, range_y=[-0.5, 1.5])

In [10]:
from tensorflow.keras.activations import relu

data = pd.DataFrame({'data': random_data, 'relu': relu(random_data)})
data.head()

,data,relu
0,-2.450118,0.0
1,-2.061522,0.0
2,-2.046942,0.0
3,-2.022047,0.0
4,-1.972046,0.0


In [ ]:
px.line(data, x='data', y='relu', width=800, range_y=[-0.5, 1.5])

In [11]:
from tensorflow.keras.activations import tanh

data = pd.DataFrame({'data': random_data, 'tanh': tanh(random_data)})
data.head()

,data,tanh
0,-2.450118,-0.985220
1,-2.061522,-0.968126
2,-2.046942,-0.967198
3,-2.022047,-0.965553
4,-1.972046,-0.961998


In [12]:
px.line(data, x='data', y='tanh', width=800, range_y=[-1.5, 1.5])

In [13]:
model = Sequential()
model.add(Dense(units=8, activation='relu', input_shape=(10,)))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### <a name='a2'></a>Kompilacja modelu
Przed rozpoczęciem trenowania sieci należy odpowiednio skonfigurować proces uczenia. W tym kroku określamy:
* rodzaj optymalizatora ([Keras - Optymalizatory](https://keras.io/optimizers/))
* funkcję straty ([Keras - Funkcje Straty](https://keras.io/losses/))
* metryki, które będziemy obserwować podczas trenowania sieci ([Keras - Metryki](https://keras.io/metrics/))

In [14]:
# klasyfikacja binarna
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# klasyfikacja wieloklasowa
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# regresja
model.compile(optimizer='rmsprop',
              loss='mse')

### <a name='a3'></a>Trenowanie modelu
Za dane wejściowe do modelu należy przekazać Numpy arrays:
* **epochs** - krotność przejścia danych przez sieć w procesie uczenia
* **batch_size** - rozmiar wsadu po którym następuje aktualizacja wag
* **validation_split** - część danych treningowych, które zostaną wykorzystane jako zbiór walidacyjny
* **validation_data** - (x_val, y_val) - dane wykorzystane do walidacji modelu

In [15]:
# model.fit(data, labels, epochs=10, batch_size=32)
# model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)
# model.fit(data, labels, epochs=10, batch_size=32, validation_data=(x_val, y_val))

### <a name='a4'></a> Przykład - klasyfikacja binarna

In [16]:
data = np.random.randn(1000, 150)
labels = np.random.randint(2, size=(1000, 1))

print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [17]:
data[:3]

array([[ 5.09876039e-01, -4.73441484e-01, -4.44413828e-01,
         1.01773000e+00,  3.98917977e-01,  5.00209603e-01,
         1.22219641e+00, -6.91195969e-01, -1.40018311e+00,
        -7.35545068e-01,  3.57010875e-01, -5.67306352e-02,
        -1.07789363e-01, -5.95897930e-02,  1.33613320e+00,
         5.93441902e-01,  4.45444737e-01, -1.18475447e+00,
        -6.53556827e-01, -1.75944730e+00,  3.21855452e-02,
        -2.52236874e-01, -1.17334716e+00, -4.36134682e-01,
        -1.37229051e+00, -9.68536165e-01,  1.57360311e+00,
        -9.17809748e-01,  1.76081276e+00,  7.04006066e-01,
         6.91361316e-02, -2.11766869e+00,  8.77060102e-01,
         4.07383777e-01,  2.53204939e-01,  2.01532647e+00,
        -1.66856473e+00,  3.59522524e-01,  2.13297495e+00,
        -7.95906693e-01,  1.97337372e+00,  9.65857273e-02,
         2.27605413e+00, -2.52172290e-01,  1.44428403e+00,
         6.49295140e-01, -2.08994545e-01, -1.10000703e-01,
        -9.50442235e-01,  1.13587367e+00,  1.03151411e+0

In [18]:
labels[:10]

array([[1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1]])

In [19]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20)

Epoch 1/20
32/32 [==============================] - 1s 2ms/step - loss: 0.8085 - accuracy: 0.5200
Epoch 2/20
32/32 [==============================] - 0s 2ms/step - loss: 0.7093 - accuracy: 0.5880
Epoch 3/20
32/32 [==============================] - 0s 2ms/step - loss: 0.6533 - accuracy: 0.6350
Epoch 4/20
32/32 [==============================] - 0s 2ms/step - loss: 0.6093 - accuracy: 0.6730
Epoch 5/20
32/32 [==============================] - 0s 2ms/step - loss: 0.5726 - accuracy: 0.7070
Epoch 6/20
32/32 [==============================] - 0s 2ms/step - loss: 0.5419 - accuracy: 0.7440
Epoch 7/20
32/32 [==============================] - 0s 2ms/step - loss: 0.5140 - accuracy: 0.7680
Epoch 8/20
32/32 [==============================] - 0s 2ms/step - loss: 0.4886 - accuracy: 0.7930
Epoch 9/20
32/32 [==============================] - 0s 2ms/step - loss: 0.4636 - accuracy: 0.8090
Epoch 10/20
32/32 [==============================] - 0s 2ms/step - loss: 0.4392 - accuracy: 0.8320
Epoch 11/20
32/32 [

In [20]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=30)

Epoch 1/20
34/34 [==============================] - 1s 2ms/step - loss: 0.8008 - accuracy: 0.5070
Epoch 2/20
34/34 [==============================] - 0s 2ms/step - loss: 0.6962 - accuracy: 0.5710
Epoch 3/20
34/34 [==============================] - 0s 2ms/step - loss: 0.6382 - accuracy: 0.6340
Epoch 4/20
34/34 [==============================] - 0s 2ms/step - loss: 0.5930 - accuracy: 0.6710
Epoch 5/20
34/34 [==============================] - 0s 2ms/step - loss: 0.5551 - accuracy: 0.7390
Epoch 6/20
34/34 [==============================] - 0s 2ms/step - loss: 0.5230 - accuracy: 0.7630
Epoch 7/20
34/34 [==============================] - 0s 2ms/step - loss: 0.4946 - accuracy: 0.7920
Epoch 8/20
34/34 [==============================] - 0s 2ms/step - loss: 0.4668 - accuracy: 0.8240
Epoch 9/20
34/34 [==============================] - 0s 2ms/step - loss: 0.4421 - accuracy: 0.8390
Epoch 10/20
34/34 [==============================] - 0s 2ms/step - loss: 0.4183 - accuracy: 0.8610
Epoch 11/20
34/34 [

In [21]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
25/25 [==============================] - 1s 12ms/step - loss: 0.7926 - accuracy: 0.4950 - val_loss: 0.7765 - val_accuracy: 0.4650
Epoch 2/20
25/25 [==============================] - 0s 4ms/step - loss: 0.7030 - accuracy: 0.5462 - val_loss: 0.7669 - val_accuracy: 0.4850
Epoch 3/20
25/25 [==============================] - 0s 4ms/step - loss: 0.6521 - accuracy: 0.6087 - val_loss: 0.7620 - val_accuracy: 0.5000
Epoch 4/20
25/25 [==============================] - 0s 3ms/step - loss: 0.6092 - accuracy: 0.6525 - val_loss: 0.7613 - val_accuracy: 0.5100
Epoch 5/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5732 - accuracy: 0.7000 - val_loss: 0.7630 - val_accuracy: 0.5200
Epoch 6/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5410 - accuracy: 0.7575 - val_loss: 0.7687 - val_accuracy: 0.5300
Epoch 7/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5123 - accuracy: 0.7675 - val_loss: 0.7754 - val_accuracy: 0.5350
Epoch 8/20
25/25 [=

In [23]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [24]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [26]:
metrics = history.history
metrics.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [28]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/20
25/25 [==============================] - 1s 10ms/step - loss: 0.8739 - accuracy: 0.5075 - val_loss: 0.7835 - val_accuracy: 0.5100
Epoch 2/20
25/25 [==============================] - 0s 4ms/step - loss: 0.7411 - accuracy: 0.5437 - val_loss: 0.7612 - val_accuracy: 0.5300
Epoch 3/20
25/25 [==============================] - 0s 5ms/step - loss: 0.6748 - accuracy: 0.5987 - val_loss: 0.7494 - val_accuracy: 0.5200
Epoch 4/20
25/25 [==============================] - 0s 3ms/step - loss: 0.6276 - accuracy: 0.6400 - val_loss: 0.7431 - val_accuracy: 0.5700
Epoch 5/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5898 - accuracy: 0.6787 - val_loss: 0.7377 - val_accuracy: 0.5500
Epoch 6/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5570 - accuracy: 0.7300 - val_loss: 0.7375 - val_accuracy: 0.5400
Epoch 7/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5280 - accuracy: 0.7550 - val_loss: 0.7393 - val_accuracy: 0.5350
Epoch 8/20
25/25 [=

In [29]:
test_data = np.random.randn(5, 150)
test_labels = np.random.randint(2, size=(5, 1))

In [30]:
model.predict(test_data)

1/1 [==============================] - 0s 87ms/step


array([[0.97115356],
       [0.6579365 ],
       [0.6401503 ],
       [0.24929969],
       [0.23309685]], dtype=float32)

In [35]:
model.predict_proba(test_data)


1/1 [==============================] - 0s 21ms/step


array([0, 0, 0, 0, 0])

In [32]:
model.predict_classes(test_data)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

### <a name='a5'></a> Przykład - klasyfikacja wieloklasowa

In [36]:
data = np.random.random((1000, 150))
labels = np.random.randint(10, size=(1000, 1))

In [37]:
print(data.shape)
print(labels.shape)

(1000, 150)
(1000, 1)


In [38]:
data[:3]

array([[1.66615406e-01, 3.42957981e-01, 9.56388501e-02, 3.14342574e-01,
        5.99003306e-01, 8.80528390e-01, 5.62339917e-01, 5.68208021e-01,
        8.81744515e-01, 3.70712795e-01, 9.01802373e-01, 4.98481480e-01,
        9.33042887e-01, 9.45387120e-01, 7.65983488e-01, 8.69979112e-01,
        3.42346013e-01, 3.70822179e-01, 9.53507573e-01, 9.27235298e-01,
        8.48992501e-02, 2.00716437e-01, 5.23994449e-01, 7.82445772e-01,
        9.55034079e-01, 5.01239808e-01, 5.60486575e-01, 7.59838361e-01,
        9.98367424e-01, 2.25178863e-01, 1.62464699e-01, 7.99779474e-01,
        3.18931492e-01, 3.69588876e-02, 8.88666535e-01, 4.54179220e-01,
        7.33929035e-01, 8.96923979e-01, 3.29385971e-01, 8.04646088e-01,
        6.96269410e-01, 8.22657196e-01, 9.52623750e-01, 6.90974011e-01,
        9.66530824e-02, 3.50261692e-01, 3.45068909e-01, 6.75940313e-01,
        3.57181490e-01, 3.68024251e-01, 5.84120870e-01, 3.25850454e-01,
        9.54322729e-01, 5.75807434e-01, 4.64705106e-02, 6.037486

In [39]:
labels[:10]

array([[2],
       [7],
       [8],
       [7],
       [0],
       [0],
       [8],
       [2],
       [1],
       [4]])

In [40]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=10)
labels

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [41]:
labels[1]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [42]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=30, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 11ms/step - loss: 2.3653 - accuracy: 0.1000 - val_loss: 2.3915 - val_accuracy: 0.0900
Epoch 2/30
25/25 [==============================] - 0s 4ms/step - loss: 2.3240 - accuracy: 0.1187 - val_loss: 2.3791 - val_accuracy: 0.0800
Epoch 3/30
25/25 [==============================] - 0s 4ms/step - loss: 2.3012 - accuracy: 0.1325 - val_loss: 2.3866 - val_accuracy: 0.0650
Epoch 4/30
25/25 [==============================] - 0s 4ms/step - loss: 2.2876 - accuracy: 0.1312 - val_loss: 2.3649 - val_accuracy: 0.0700
Epoch 5/30
25/25 [==============================] - 0s 3ms/step - loss: 2.2751 - accuracy: 0.1587 - val_loss: 2.3764 - val_accuracy: 0.0850
Epoch 6/30
25/25 [==============================] - 0s 4ms/step - loss: 2.2617 - accuracy: 0.1550 - val_loss: 2.3902 - val_accuracy: 0.0650
Epoch 7/30
25/25 [==============================] - 0s 4ms/step - loss: 2.2512 - accuracy: 0.1450 - val_loss: 2.3627 - val_accuracy: 0.0850
Epoch 8/30
25/25 [=

In [43]:
test_data = np.random.random((10, 150))

model.predict(test_data)

1/1 [==============================] - 0s 59ms/step


array([[0.07369861, 0.09249242, 0.18032084, 0.03059305, 0.06587502,
        0.08483427, 0.09748222, 0.14116094, 0.14901581, 0.08452677],
       [0.07990202, 0.11647557, 0.06471377, 0.07190457, 0.09419192,
        0.08015964, 0.08018229, 0.11712866, 0.16833752, 0.12700407],
       [0.07479502, 0.07221207, 0.15478079, 0.05738892, 0.08875635,
        0.10462207, 0.16037598, 0.05191165, 0.18807608, 0.04708109],
       [0.05319747, 0.14331974, 0.07122316, 0.08525889, 0.09124152,
        0.07999104, 0.07774187, 0.05058277, 0.18624386, 0.16119967],
       [0.04981272, 0.02678962, 0.07167672, 0.03423252, 0.17900136,
        0.04873646, 0.24025975, 0.0949509 , 0.13110553, 0.12343442],
       [0.10079993, 0.04206309, 0.04416388, 0.23134512, 0.1034952 ,
        0.06383339, 0.04225197, 0.11201251, 0.1128263 , 0.14720865],
       [0.02628751, 0.13555218, 0.02898932, 0.08784746, 0.08243381,
        0.04149484, 0.02035956, 0.20060265, 0.291119  , 0.08531362],
       [0.12937054, 0.10474302, 0.1519819

In [46]:
predictions = model.predict(test_data)

# Znalezienie indeksów największych prawdopodobieństw, które odpowiadają klasom
predicted_classes = np.argmax(predictions, axis=1)

predicted_classes

1/1 [==============================] - 0s 22ms/step


array([2, 8, 8, 8, 6, 3, 8, 6, 2, 3])

In [44]:
model.predict_classes(test_data)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

### <a name='a6'></a> Przykład - regresja

In [47]:
data = np.random.random((1000, 150))
labels = 50 * np.random.random(1000)

In [51]:
data[:3]

array([[3.04510209e-01, 8.51026939e-01, 6.49757460e-01, 8.44554309e-01,
        6.04334665e-01, 2.09615682e-01, 9.15372454e-01, 8.42940522e-01,
        3.27617994e-01, 1.65437009e-01, 9.92930209e-01, 2.48390304e-01,
        5.69328333e-01, 9.93862750e-01, 2.47287040e-01, 7.19710158e-01,
        4.48788725e-01, 6.01764258e-01, 6.09636507e-01, 8.18205421e-01,
        6.74416723e-01, 6.79383021e-01, 5.07329784e-01, 5.01815868e-01,
        4.43599674e-01, 8.36272910e-02, 2.63451181e-01, 1.37900369e-01,
        9.81954679e-02, 5.53203150e-04, 8.25878545e-01, 1.30309629e-01,
        7.44254570e-01, 1.72983434e-01, 4.23950614e-01, 7.84301262e-01,
        6.65285474e-01, 1.13916130e-01, 2.67698677e-01, 8.37625638e-01,
        7.39409651e-01, 4.32974098e-02, 7.75985950e-01, 7.03996673e-01,
        5.61410705e-01, 9.90173065e-01, 9.34849196e-01, 1.41919868e-01,
        2.96477630e-01, 2.82044138e-01, 2.94888461e-01, 6.88853475e-01,
        9.33428106e-01, 2.61598560e-01, 1.59871677e-01, 4.315873

In [50]:
labels[:10]

array([15.76496735, 16.4493407 ,  2.32671486, 30.18020031, 10.97452681,
       39.80629216, 47.52505672,  9.52026182,  6.72048121, 28.84454747])

In [52]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mse')

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 8ms/step - loss: 637.2654 - val_loss: 535.8581
Epoch 2/30
25/25 [==============================] - 0s 4ms/step - loss: 405.0469 - val_loss: 337.9968
Epoch 3/30
25/25 [==============================] - 0s 3ms/step - loss: 262.4178 - val_loss: 227.9881
Epoch 4/30
25/25 [==============================] - 0s 4ms/step - loss: 209.3440 - val_loss: 200.4864
Epoch 5/30
25/25 [==============================] - 0s 4ms/step - loss: 203.1005 - val_loss: 198.8155
Epoch 6/30
25/25 [==============================] - 0s 4ms/step - loss: 202.4567 - val_loss: 200.4423
Epoch 7/30
25/25 [==============================] - 0s 4ms/step - loss: 201.9531 - val_loss: 198.7902
Epoch 8/30
25/25 [==============================] - 0s 3ms/step - loss: 201.3864 - val_loss: 200.7749
Epoch 9/30
25/25 [==============================] - 0s 5ms/step - loss: 200.9088 - val_loss: 200.6030
Epoch 10/30
25/25 [==============================] - 0s 4ms/step - loss: 200.0170 

In [53]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
25/25 [==============================] - 1s 15ms/step - loss: 21.3177 - mse: 643.8987 - val_loss: 19.4888 - val_mse: 529.3600
Epoch 2/30
25/25 [==============================] - 0s 6ms/step - loss: 16.0944 - mse: 391.3336 - val_loss: 15.0880 - val_mse: 313.4435
Epoch 3/30
25/25 [==============================] - 0s 5ms/step - loss: 13.2284 - mse: 247.6106 - val_loss: 12.8862 - val_mse: 224.2166
Epoch 4/30
25/25 [==============================] - 0s 5ms/step - loss: 12.3509 - mse: 207.3120 - val_loss: 12.0772 - val_mse: 200.7880
Epoch 5/30
25/25 [==============================] - 0s 7ms/step - loss: 12.2800 - mse: 202.9640 - val_loss: 12.0872 - val_mse: 201.0865
Epoch 6/30
25/25 [==============================] - 0s 6ms/step - loss: 12.2620 - mse: 202.5933 - val_loss: 12.1068 - val_mse: 201.6442
Epoch 7/30
25/25 [==============================] - 0s 6ms/step - loss: 12.2385 - mse: 201.9897 - val_loss: 12.1580 - val_mse: 203.1640
Epoch 8/30
25/25 [=============================

In [54]:
test_data = np.random.random((10, 150))

model.predict(test_data)

1/1 [==============================] - 0s 54ms/step


array([[21.160915],
       [24.61333 ],
       [26.216751],
       [23.621943],
       [22.059143],
       [26.530437],
       [22.137417],
       [26.532566],
       [18.796711],
       [27.66217 ]], dtype=float32)